In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go

from binary_btsp_network_torch import BinaryBTSPNetworkTorch
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---------------- 参数设置 ----------------
n_input = 10000       # CA3 神经元数量
n_memory = 15000       # CA1 神经元数量
fq = 0.0025             # BTSP 更新概率
fp = 0.005             # 输入稀疏度
fw = 0.6           # 连接稀疏度
batch_size = 256
n_repeats = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

# 记忆规模与掩码比例
memory_sizes = np.arange(1000,13000,1000)
mask_ratios = np.linspace(0, 1.0, 10)

# 结果矩阵 (行: 记忆规模, 列: 掩码比例)
results = np.zeros((len(memory_sizes), len(mask_ratios)))

print("计算不同记忆规模与掩码比例下的平均归一化海明距离...\n")

# ---------------- 主循环 ----------------
for i, n_pat in enumerate(tqdm(memory_sizes, desc="Memory sizes")):
    # 构建网络
    net = BinaryBTSPNetworkTorch(
        n_input=n_input,
        n_memory=n_memory,
        fq=fq,
        fp=fp,
        fw=fw,
        device=device,
        threshold=8
    )

    # 生成输入模式
    patterns = net.make_sparse_patterns(n_pat)

    # 训练网络
    net.train_patterns(patterns, batch_size=batch_size)

    # 评估记忆保留效果
    mean_rel_diss = net.compute_relative_dissimilarity_vectorized(
        patterns=patterns,
        mask_ratios=mask_ratios,
        n_repeats=n_repeats,
        batch_size=batch_size,
    )

    results[i, :] = mean_rel_diss


计算不同记忆规模与掩码比例下的平均归一化海明距离...



Memory sizes: 100%|██████████| 12/12 [09:22<00:00, 46.91s/it]


In [3]:

# ---------------- 绘制结果 ----------------
X, Y = np.meshgrid(mask_ratios, memory_sizes)
Z = results

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])
fig.update_layout(
    title="不同记忆规模与掩码比例下的记忆保留效果",
    scene=dict(
        xaxis_title="掩码比例",
        yaxis_title="记忆模式数量",
        zaxis_title="平均归一化海明距离",
        zaxis=dict(range=[0, 2])
    ),
    width=800,
    height=600
)

fig.show()
